In [1]:
import requests
import json
import psycopg2
import pandas as pd 
import sqlalchemy as salc
import numpy as np
# import seaborn as sns
# import matplotlib.pyplot as plt
import warnings
import datetime
import json
warnings.filterwarnings('ignore')

with open("config.json") as config_file:
    config = json.load(config_file)

database_connection_string = 'postgres+psycopg2://{}:{}@{}:{}/{}'.format(config['user'], config['password'], config['host'], config['port'], config['database'])

dbschema='augur_data'
engine = salc.create_engine(
    database_connection_string,
    connect_args={'options': '-csearch_path={}'.format(dbschema)})

# Database Connection

In [70]:
github_id = input("GitHub ID")

QUERY = """
select distinct repo_id from 
(
select contributors.cntrb_id, pull_requests.pr_augur_contributor_id, pull_requests.repo_id 
from contributors, pull_requests
where contributors.cntrb_id=pull_requests.pr_augur_contributor_id
and gh_login='%s') a 

""" % github_id

repo_id = pd.read_sql(QUERY, con=engine)
repo_list = repo_id['repo_id'].tolist()

GitHub ID dpose


In [71]:
placeholders= ', '.join(['%s']*len(repo_list))  # "%s, %s, %s, ... %s"
SQL = """
SELECT
    programming_language,
    COUNT ( * ) AS language_files 
FROM
    repo_labor 
WHERE
    repo_id IN ({}) 
GROUP BY
    repo_id,
    programming_language 
ORDER BY
    language_files DESC;
""".format(placeholders)

SQL = SQL % tuple(repo_list)

language_list = pd.read_sql(SQL, con=engine, params=repo_list)
languages = language_list.head(2)['programming_language'].tolist()
display(languages)

['JavaScript', 'TypeScript']

In [72]:
languages[0]

'JavaScript'

In [74]:
url = "https://job-search4.p.rapidapi.com/simplyhired/"
headers = {
    'x-rapidapi-key': "0a1449c97fmshc98d54f2a209dfbp175d54jsn1408a1529a02",
    'x-rapidapi-host': "job-search4.p.rapidapi.com"
}
trend_dict = dict()

In [75]:
# job_listing_search = input("Type of Job:")
job_listing_search = languages[0] + " Language"

if job_listing_search in trend_dict:
    trend_dict[job_listing_search] += 1
else:
    trend_dict[job_listing_search] = 1

# page_number = "-1" page: page_number
querystring = {"query": job_listing_search}

response = requests.request("GET", url, headers=headers, params=querystring)
result = json.loads(response.text)


job_listing_results = result['results']
print("The search returned " + str(len(job_listing_results))  + " results") 


The search returned 19 results


In [76]:
for each_job in job_listing_results:
    print("Job Title: "  + each_job["title"])
    print("Company Name: "  + each_job["company_name"])
    print("Location: "  + each_job["location"])
    print("Description: "  + each_job["description"])
    print("Date posted: "  + each_job["date_posted"])
    print("url : "  + each_job["detail_url"])
    print('\n---------------------------------------------------------------------------------------------------\n')


Job Title: Junior JavaScript Developer
Company Name: Sage Digital
Location: Remote
Description: Experience with modern databases and query languages, like PostgreSQL, MongoDB, GraphQL, etc. Bootcamp, bachelor’s degree, or equivalent experience.
Date posted: 2021-04-16T00:00:00
url : https://www.simplyhired.com/job/plGdjQGOZNKNF45w-s5XqB9QNCrpk-m0MS6G7Ngn8swxVmJgC2emOQ?q=JavaScript+Language

---------------------------------------------------------------------------------------------------

Job Title: Senior Backend Developer
Company Name: Trameter
Location: Denver, CO
Description: 3+ years of experience and strong coding skills in developing scalable and performant software systems in at least one modern backend programming language like…
Date posted: 2021-04-23T00:00:00
url : https://www.simplyhired.com/job/_GEuLe_2cIovmMD8weVOhp1P3i0oZd9SEkwC8a8PYRg3dSaNOoIfPQ?q=JavaScript+Language

---------------------------------------------------------------------------------------------------

J

In [17]:
trend_dict

{'University of Missouri': 1}

In [20]:
result['results']

[{'title': 'Retail Associate',
  'date_posted': '2021-04-19T00:00:00',
  'location': 'Panama City, FL',
  'country': 'United States',
  'state': 'Florida',
  'city': 'Panama City',
  'description': 'We are currently accepting applications for full-time and part-time positions. Applicants must be available to work some nights and weekends.',
  'detail_url': 'https://www.simplyhired.com/job/KFe-Gq1JAPocwLrJa5XYZurOA4gRH4a4xWTN79IQ7kRbnXAOzrKzxA?q=Panama',
  'source': 'lawjobs.com',
  'company_name': 'Hobby Lobby'},
 {'title': 'Personal Assistant/Caregiver/CNA',
  'date_posted': '2021-04-16T00:00:00',
  'location': 'Panama City Beach, FL',
  'country': 'United States',
  'state': 'Florida',
  'city': 'Panama City Beach',
  'description': 'Supports our assisted living and memory care residents by providing companionship, supportive listening and activities of daily living, i.e. bathing, dressing,…',
  'detail_url': 'https://www.simplyhired.com/job/XXsg6L5ZDeAmvZ0zK5ps8e9_4blF9WFnTa-TGQi9hS